In [ ]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Reading Dataset

In [ ]:
black_list_df = pd.read_csv("dataset/black_list.csv",names=['blacklist'])
words_df = pd.read_csv("dataset/spam_words.csv",names=['spam_words'])
dataset = pd.read_csv('dataset/spam_ham_dataset.csv')

## Preprocessing dataset

### A. Black list dataset

In [ ]:
black_list = black_list_df['blacklist'].tolist()

In [ ]:
def process_email(email_id):
    email_id = email_id.replace('@','')
    email_id = email_id.replace('.','')
    email_id = email_id.replace('+','')
    email_id = email_id.replace('/','')
    email_id = email_id.replace('-','')
    email_id = email_id.replace('"','')
    return email_id

n = len(black_list)
for i in range(n):
    black_list[i] = process_email(black_list[i])

white_list = []

### B. Spam words dataset

In [ ]:
from nltk.tokenize import RegexpTokenizer
def clean_str(string, reg = RegexpTokenizer(r'[a-z]+')):
    string = string.lower()
    tokens = reg.tokenize(string)
    return " ".join(tokens)

words_df['spam_words'] = words_df['spam_words'].apply(lambda string: clean_str(string))

words_df

,spam_words
0,text
1,hidden assets
2,risk
3,free
4,more
...,...
571,your income
572,your status
573,zero chance
574,zero percent


In [ ]:
nwords = words_df['spam_words'].tolist()
spam_words = [i for i in nwords if len(i)>=5]

### C. Spam-Ham dataset

In [ ]:
dataset['text'] = dataset['text'].apply(lambda string: clean_str(string))

In [ ]:
from nltk.corpus import stopwords

def remove_stopwords(text):  
    stop_words = stopwords.words('english')
    stop_words.append('subject')
    words = text.lower().split()
    words = [word for word in words if word not in stop_words and len(word) >= 5]    
    text_new = ' '.join(words)
    
    return text_new

dataset['text'] = dataset.text.apply(remove_stopwords)
dataset.head(5)

,Unnamed: 0,label,text,label_num
0,605,ham,enron methanol meter follow monday preliminary...,0
1,2349,ham,january attached hplnol hplnol,0
2,3624,ham,retreat around wonderful leaders retreat extre...,0
3,4685,spam,photoshop windows office cheap trending abasem...,1
4,2030,ham,indian springs revenue understanding sends che...,0


In [ ]:
spam_df = pd.read_csv('dataset/spam.csv')
ham_df = pd.read_csv('dataset/ham.csv')

text = dataset['text'].tolist()
label = dataset['label_num'].tolist()
words = spam_df['word'].tolist()
p_ham = ham_df['score'].tolist()
p_spam = spam_df['score'].tolist()

prob_spam = {}
prob_ham = {}

n1 = len(words)
for i in range(0,n1):
    words[i] = str(words[i])
    prob_spam[words[i]] = p_spam[i]
    prob_ham[words[i]] = p_ham[i]

## Rabin-Karp algorithm 

### A. Implementation

In [ ]:
def rabin_karp_match(string, string_list, modulo_number):
    total = 256
    n = len(string_list) # length of the string list
    string = str(string) 
    l = len(string) # string length
    k=0
    for k in range(n):
        text = string_list[k] # element of the string_list
        N=len(text)
        i = 0
        j = 0
        s_num = 0   
        t_num = 0    
        hash_f = 1
        flag=0
        if(N==l):
            for i in range(l-1):
                hash_f = (hash_f * total)% modulo_number
            for i in range(l):
                s_num = (total * s_num + ord(string[i]))% modulo_number
                t_num = (total * t_num + ord(text[i]))% modulo_number
            for i in range(N-l + 1):
                if s_num == t_num:
                    for j in range(l):
                        if text[i + j] != string[j]:
                            break  
                    j+= 1          
                    if j == l:
                        return 1
                if i < N-l:
                    t_num = (d*(t_num-ord(text[i])*hash_f) + ord(text[i + l]))% modulo_number
                    if t_num < 0:
                        t_num = t_num + modulo_number
    return 0

def rabin_karp_pattern(pattern, text, modulo_number):
    total = 256
    pattern = str(pattern)
    M = len(pattern) # pattern length
    N = len(text) # text length
    i = 0
    j = 0
    p_num = 0   
    t_num = 0    
    hash_f = 1
    flag=0

    for i in range(M-1):
        hash_f = (hash_f * total)% modulo_number
    for i in range(M):
        p_num = (total * p_num + ord(pattern[i]))% modulo_number
        t_num = (total * t_num + ord(text[i]))% modulo_number
    for i in range(N-M + 1):
        if p_num == t_num:
            for j in range(M):
                if text[i + j] != pattern[j]:
                    break
  
            j+= 1
            if j == M:
                return 1
        if i < N-M:
            t_num = (total*(t_num-ord(text[i])*hash_f) + ord(text[i + M]))% modulo_number
            if t_num < 0:
                t_num = t_num + modulo_number

    return 0

### B. Black list

In [ ]:
def check_in_email(sender,email_list):
  return rabin_karp_match(sender,email_list,101)

## KMP algorithm

### A. Implementation

In [ ]:
def computeLPSArray(pat, M, lps):
    len = 0
    lps[0] = 0
    i = 1
    while i < M:
        if pat[i] == pat[len]:
            len += 1
            lps[i] = len
            i += 1
        else:
            if len != 0:
                len = lps[len - 1]
            else:
                lps[i] = 0
                i += 1

def KMP(pat, txt):
    M = len(pat)
    N = len(txt)
    count = 0
    lps = [0] * M
    j = 0 
    computeLPSArray(pat, M, lps)
    i = 0 
    while i < N:
        if pat[j] == txt[i]:
            i += 1
            j += 1
        if j == M:
            count += 1
            j = lps[j - 1]
        elif i < N and pat[j] != txt[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return count

### C. Word based method

In [ ]:
def check_spam_text_message(words,text):
  text = clean_str(text)
  for i in range(len(words)):
    if(KMP(words[i],text)!=0):
      return True
  return False

## Compressed Tries

### A. Implementation

In [ ]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_end = True

class Trie:
    def __init__(self):
        #root node in trie
        self.root = self.getNewNode()

    def getNewNode(self):
        return TrieNode()

    def insert(self,word):
        present_node = self.root
        length = len(word)    
        i = 0
        tt = False
        while i < (length):
            # if current character is not present
            children = list(present_node.children.keys())
            index = -1
            if tt:
                print(children)
            for j in range(len(children)):
                stri = children[j]
                #print(stri,end='\n')
                if(stri[0] == word[i]):
                    index = j
                    break
      
            if index == -1:
                present_node.children[word[i:]] = self.getNewNode()
                break

            else :
                #gather path starting from word[i]
                pointe = children[index]#already existing path
                #word[i:] needed to be inserted
                j = 0
                k = i
                flag = False
                while j < (len(pointe)):
                    if(word[k] != pointe[j]):
                        paths = list(present_node.children.keys())
                        same_node = pointe[:j]
                        present_node.children[same_node] = self.getNewNode()

                        temp_node = present_node.children[pointe]
                        del present_node.children[pointe]
            
                        present_node = present_node.children[same_node]
                        present_node.children[pointe[j:]] = temp_node
                        present_node.children[word[k:]] = self.getNewNode()

                        #print(paths)
                        #print("Inserted2 {}".format(word[k:]))
                        flag = True
                        break
                    j += 1
                    k += 1
                if(flag) :
                    break
                else:
                    present_node = present_node.children[pointe]
                    i = k-1
            i += 1

    def search(self, pattern):
        pattern = str(pattern)
        present_node = self.root
        length = len(pattern)
        i = 0
        boo = False
        while i < length:
            paths = list(present_node.children.keys())
            a = present_node.children.get(pattern[i])
            l= len(pattern[i:])
            b = None
            nex = pattern[i:]
            k = len(pattern[i:])
            while(k>=0):
                b = present_node.children.get(pattern[i:i+k])
                nex = pattern[i:i+k]
                if not b == None:
                    break
                k -= 1
            if a==b:
                b = None
            if (a == None and b == None ):
                return False
            elif b == None :
                if pattern[i] == '$':
                    return True
                present_node = present_node.children[pattern[i]]
                i += 1
            else:
                if nex == pattern[i:]:
                    return True
                else:
                    present_node = present_node.children[nex]
                    i += len(nex)
        return False

In [ ]:
def classify_tries(message,words_trie, prob_spam, prob_ham):
    ps = 0.2864394488759971 
    ph = 0.7135605511240029

    message = message.split(' ')
    
    for word in message:
      if words_trie.search(word+'$'):
        ps *= prob_spam[word]
        ph *= prob_ham[word]
    print(ps, ph)
    if ps == 0 or ph == 0 :
      rp_spam = 0
    else:
      rp_spam = ps/(ps+ph)
    print(rp_spam)
    if(rp_spam>=0.95):
        return True
    else:
        return False

In [ ]:
words_trie = Trie()
for word in words:
  words_trie.insert(word+'$')

# Classification

In [ ]:
email_id = input("Enter email address : ")
email_content = input("Enter email content : ")

if check_in_email(email_id,black_list):
  print("Spam, email id in blacklist")
elif check_in_email(email_id,white_list):
  print("Ham, email id in White list")
elif check_spam_text_message(spam_words,email_content):
  print("Spam, email content seems to be spam by word-based")
  black_list.append(process_email(email_id))
elif classify_tries(email_content, words_trie, prob_spam, prob_ham):
  print("Spam, email content seems to be spam by Heuristic method")
  black_list.append(process_email(email_id))
elif not check_in_email(email_id,white_list):
  print("Ham")
  white_list.append(process_email(email_id))
else:
  print("Ham")